<a href="https://colab.research.google.com/github/Sunnychin1027/Deep_Learning-RNN-and-Pytorch/blob/main/my_pytorch_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SC201L16


In [2]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [3]:
# Load Existing Dataset
#####################################
import torchvision.datasets as dset
import torchvision.transforms as T
train_data = dset.MNIST(root='./train', train=True, download=True, transform = T.ToTensor())
val_data = dset.MNIST(root='./val', train=False, download=True, transform = T.ToTensor())
#####################################

In [4]:
# Check Data Dimension
#####################################
print(train_data)
print(val_data)
#####################################

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./val
    Split: Test
    StandardTransform
Transform: ToTensor()


In [5]:
print(train_data[0])

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [6]:
train_data[0][0].shape
# 1 = inchannel grayscale

torch.Size([1, 28, 28])

In [7]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


In [8]:
from torch.nn.modules.pooling import MaxPool2d
# Build Model
import torch.nn as nn
# model = nn.Sequential(
#     # N x 1 x 28 x 28
#     nn.Conv2d(1, 64, 3, 1, 1),
#     nn.BatchNorm2d(64),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     # N x 64 x 14 x 14
#     nn.Flatten(),
#     nn.Linear(64 * 14 * 14, 10)
# )


# class MyModel(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv2d = nn.Conv2d(1, 64, 3, 1, 1)
#     self.b_n_2d = nn.BatchNorm2d(64)

#     self.con2d2 = nn.Conv2d(64, 128, 3, 1, 1)
#     self.b_n_2d2 = nn.BatchNorm2d(128)

#     self.fc = nn.Linear(128*14*14, 10)
#   def forward(self, x):
#     out = self.conv2d(x)
#     out = self.b_n_2d(out)
#     out = nn.functional.relu(out)

#     out = self.con2d2(out)
#     out = self.b_n_2d2(out)
#     out = nn.functional.relu(out)

#     out = nn.functional.max_pool2d(out, (2,2))
#     out = torch.flatten(out, 1)
#     out = self.fc(out)
#     return out
# model = MyModel()

class MyRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    output, (h_n, c_n) = self.lstm(x)
    out = output[:, -1, :] # the last h_n
    # out = h_n.squeeze() It can also be written like this, replacing the previous line
    print(f"out.shape: {out.shape}, / h_n.shape: {h_n.shape}")
    out = self.fc(out)
    return out

model = MyRNN(28, 64, 10)

In [9]:
# Move model to GPU
model = model.cuda()

In [10]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [11]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      x = x.squeeze()# Avoid error: LSTM expected to be 2D or 3D tensor
      scores = model(x) # Calling forward(self, x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [12]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      x = x.squeeze()
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [13]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 128, 64])
out.shape: torch.Size([128, 64]), / h_n.shape: torch.Size([1, 12